In [1]:
import lancedb
import torch
from lancedb.pydantic import LanceModel, Vector
from lancedb.embeddings import get_registry

In [2]:
db = lancedb.connect("/tmp/db")
model = get_registry().get("sentence-transformers").create(name="all-MiniLM-L12-v2", device="cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")

In [3]:
class Words(LanceModel):
    text: str = model.SourceField()
    vector: Vector(model.ndims()) = model.VectorField()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/opt/miniconda3/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
table = db.create_table("words", schema=Words)
table.add(
    [
        {"text": "hello world"},
        {"text": "goodbye world"}
    ]
)

In [5]:
query = "greetings"
actual = table.search(query).limit(1).to_pydantic(Words)[0]
print(actual.text)

hello world


In [11]:
db["words"].head()

pyarrow.Table
text: string not null
vector: fixed_size_list<item: float>[384] not null
  child 0, item: float
----
text: [["hello world","goodbye world"]]
vector: [[[-0.07597318,-0.005261974,0.011456312,-0.06798462,-0.0030688604,...,0.05992768,-0.023471534,-0.05176601,-0.011127043,0.034954645],[-0.010689703,0.0022895744,-0.003602911,-0.07726417,0.024333011,...,0.040215474,0.03609358,-0.05018321,-0.026402961,0.05592772]]]